## Sentiment analysis
Emotions are one of the important aspects of politicians' speeches. It would be interesting to identify whether a certain quotation carries negative or positive emotions. This notebook aims to achieve that by performing sentiment analysis on the quotations.


In [3]:
# load the dataset
import pandas as pd
data = pd.read_pickle("../data/binary/us-politicians.pickle")
data.head()

,speaker_id,quote_id,quotation,speaker,party,Date-Time
0,1282411,2015-06-11-000090,25-year cost estimate for the U.S. nuclear mod...,Ed Markey,29552,2015-06-11
1,1282411,2015-08-11-090504,The state is transitioning to a higher percent...,Ed Markey,29552,2015-08-11
2,1282411,2015-10-27-125840,This highly coveted TIGER grant funding would ...,Ed Markey,29552,2015-10-27
3,1282411,2015-01-19-008025,carefully cutting the Gordian knot that has ti...,Ed Markey,29552,2015-01-19
4,1282411,2015-12-09-094134,The amendments would improve the safety of agi...,Ed Markey,29552,2015-12-09


In [ ]:
democratic_party = 29552
republican_party = 29468
democrats = 